In [8]:
from src.data_loader import PVDataLoader, PVDataGenerator
from src.data_loader import train_test_split

In [33]:
# data parameters
DAY_INIT = 0
DAY_MIN = 8
DAY_MAX = 16
N_DAYS = 5
N_DAYS_TRAIN = 5
MINUTE_INTERVAL = 5
DAILY_DATA_POINTS = (DAY_MAX - DAY_MIN) * 60 / MINUTE_INTERVAL
N_HOURS_PRED = 2
N_SYSTEMS = 1
RADIUS = 0.35
CIRCLE_COORDS = (55, -1.5)
# create a tuple of 4 coordinates that form a polygon in the uk

POLY_COORDS = ((50, -6), (50.5, 1.9), (57.6, -5.5), (58, 1.9))

In [34]:
loader = PVDataGenerator(n_days=N_DAYS,
                    day_init=DAY_INIT,
                    n_systems=N_SYSTEMS,
                    coords=CIRCLE_COORDS,
                    minute_interval=MINUTE_INTERVAL,
                    day_min=DAY_MIN,
                    day_max=DAY_MAX,
                    folder_name='pv_data',
                    file_name_pv='pv_data_clean.csv',
                    file_name_location='location_data_clean.csv',
                    distance_method='circle',
                    drop_nan=False)

X, y = loader.get_time_series()

==> Loading data
==> Loaded data in: 0 m : 10 sec

==> Loading data
==> Loaded data in: 0 m : 0 sec



In [35]:
x_train, y_train, x_test, y_test = train_test_split(X, y, minute_interval=MINUTE_INTERVAL, n_hours=N_HOURS_PRED)

In [36]:
from src.baselines import (ExponentialSmoothingModel, 
                           AutoRegressionModel, 
                           ARIMAModel, 
                           SARIMAXModel, 
                           MarkovAutoregressionModel)

In [37]:
y_train, y_test = y_train.numpy(), y_test.numpy()

In [125]:
kwargs = {'trend': 'mul', 'seasonal': 'add', 'seasonal_periods': N_DAYS, 'use_boxcox': True}

In [126]:
exp_model = ExponentialSmoothingModel(y_train=y_train, y_test=y_test, **kwargs)

In [130]:
fit = exp_model.model.fit()
simulations = fit.simulate(len(y_test), repetitions=100, error='mul')

/Users/eirikbaekkelund/opt/anaconda3/envs/gp/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [131]:
y_pred = exp_model.model.predict(params=fit.params, start=len(y_train), end=len(y_train) + len(y_test)-1)